In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터셋 전처리 -> DataFrame으로 합치느라 램이 터짐
/content/drive/MyDrive/DILAB/MARS/UMLS/META/

## RRF파일 로드

In [ ]:
UMLS_DIR = "/content/drive/MyDrive/DILAB/MARS/UMLS/META"

import pandas as pd

# MRCONSO: 개념명
mrconso = pd.read_csv(f"{UMLS_DIR}/MRCONSO.RRF", sep="|", header=None, dtype=str)
mrconso = mrconso[[0, 1, 11, 14]]  # CUI, LAT, SAB, STR
mrconso.columns = ["CUI", "LAT", "SAB", "STR"]

# MRDEF: 정의(설명)
mrdef = pd.read_csv(f"{UMLS_DIR}/MRDEF.RRF", sep="|", header=None, dtype=str)
mrdef = mrdef[[0, 5]]  # CUI, DEF
mrdef.columns = ["CUI", "DEF"]

# MRSTY: semantic type
mrsty = pd.read_csv(f"{UMLS_DIR}/MRSTY.RRF", sep="|", header=None, dtype=str)
mrsty = mrsty[[0, 3]]  # CUI, STY
mrsty.columns = ["CUI", "STY"]

# MRREL: 관계 정보
# MRREL 정식 16개 + 트레일러용 더미 1개 = 17개 이름 지정
names17 = ["CUI1","AUI1","STYPE1","REL","CUI2","AUI2","STYPE2","RELA",
           "RUI","SRUI","SAB","SL","RG","DIR","SUPPRESS","CVF","_TRAIL"]

mrrel = pd.read_csv(
    f"{UMLS_DIR}/MRREL.RRF",
    sep="|", header=None, names=names17, dtype=str,
    engine="python", keep_default_na=False
)

# 마지막 더미 컬럼 제거
mrrel = mrrel.drop(columns=["_TRAIL"])

# 이제 RELA가 진짜 'treats/may_treat/...' 같은 문자열이어야 합니다.
print("RELA 상위 30개:\n", pd.Series(mrrel["RELA"]).value_counts().head(30))
print("SAB 상위 10개:\n", pd.Series(mrrel["SAB"]).value_counts().head(10))


RELA 상위 30개:
 RELA
                             11859344
inverse_isa                   4091859
isa                           4091859
translation_of                3039107
has_translation               3039107
has_member                    2271320
member_of                     2271320
classifies                    2146047
classified_as                 2146047
has_inactive_ingredient       1643582
inactive_ingredient_of        1643582
has_finding_site               918107
finding_site_of                918107
mapped_to                      824094
mapped_from                    824094
has_method                     695592
method_of                      695592
has_associated_morphology      647949
associated_morphology_of       647949
has_expanded_form              508576
expanded_form_of               508576
active_ingredient_of           406281
has_active_ingredient          406281
concept_in_subset              401480
subset_includes_concept        401480
has_component                  

## 질병/약물 CUI 집합 만들기(STY 기준, 포괄적으로)

In [ ]:
# 질병/약물로 볼 Semantic Type 세트 (필요에 따라 줄이거나 늘리세요)
DISEASE_STYS = {
    "Disease or Syndrome", "Neoplastic Process", "Mental or Behavioral Dysfunction",
    "Pathologic Function", "Injury or Poisoning", "Congenital Abnormality",
    "Acquired Abnormality", "Finding"  # Finding은 많으면 제외해도 됨
}
DRUG_STYS = {
    "Pharmacologic Substance", "Clinical Drug", "Organic Chemical",
    "Antibiotic", "Biomedical or Dental Material", "Immunologic Factor",
    "Hormone", "Vitamin", "Hazardous or Poisonous Substance"
}

disease_cuis = set(mrsty[mrsty["STY"].isin(DISEASE_STYS)]["CUI"].unique())
drug_cuis    = set(mrsty[mrsty["STY"].isin(DRUG_STYS)]["CUI"].unique())

len(disease_cuis), len(drug_cuis)


(679345, 619651)

## 치료/적응증 계열 RELA만 추출 (양방향 모두 고려)

In [ ]:
# 당신 데이터의 RELA 목록을 보고 실제 값으로 조정하세요
TREAT_RELAS = {
    "treats", "may_treat", "has_indication", "has_therapeutic_effect",
    "prevents", "may_prevent"
}

# (질병→약물)
rel1 = mrrel[(mrrel["RELA"].isin(TREAT_RELAS)) &
             (mrrel["CUI1"].isin(disease_cuis)) &
             (mrrel["CUI2"].isin(drug_cuis))] \
        .rename(columns={"CUI1":"disease_cui","CUI2":"drug_cui"})

# (약물→질병)
rel2 = mrrel[(mrrel["RELA"].isin(TREAT_RELAS)) &
             (mrrel["CUI1"].isin(drug_cuis)) &
             (mrrel["CUI2"].isin(disease_cuis))] \
        .rename(columns={"CUI2":"disease_cui","CUI1":"drug_cui"})

rels = pd.concat([rel1, rel2], ignore_index=True).drop_duplicates()
print("관계 수:", len(rels))


관계 수: 16007


## 이름(ENG)·설명 붙이기

In [ ]:
# 이름: MRCONSO에서 ENG만 사용
names = mrconso[mrconso["LAT"]=="ENG"][["CUI","STR"]]

# 설명: MRDEF는 일부 CUI에만 있으므로 LEFT JOIN 유지
mrdef_ = mrdef.rename(columns={"DEF":"disease_description"})

out = (
    rels
    .merge(names, left_on="disease_cui", right_on="CUI", how="left")
    .rename(columns={"STR":"disease_name"}).drop(columns=["CUI"])
    .merge(mrdef_, left_on="disease_cui", right_on="CUI", how="left")
    .drop(columns=["CUI"])
    .merge(names, left_on="drug_cui", right_on="CUI", how="left")
    .rename(columns={"STR":"drug_name"}).drop(columns=["CUI"])
)

# 열 정리
out = out[["disease_cui","disease_name","disease_description","drug_cui","drug_name","RELA","SAB"]]
print(out.head(), "\n총 건수:", len(out))


## 매핑 결과를 CSV로 저장

In [4]:
SAVE = "/content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs.csv"
out.to_csv(SAVE, index=False, encoding="utf-8")
print("✅ 저장:", SAVE)


NameError: name 'out' is not defined

# DuckDB로 파일 위에서 조인

In [2]:
!pip -q install duckdb

import os, gc, duckdb

# ▶ UMLS 파일 경로 (본인 경로로 유지)
UMLS_DIR = "/content/drive/MyDrive/DILAB/MARS/UMLS/META"

# ▶ 임시 CSV 경로 (라인-split로 생성)
MRCONSO_PREF_TMP = "/content/mrconso_pref_tmp.csv"  # CUI, STR (ENG + 대표명만)
MRDEF_TMP        = "/content/mrdef_min_tmp.csv"     # CUI, DEF (스마트 추출)

# ▶ DuckDB 파일 DB (세션 재시작/셀 분리에도 보존)
DB_PATH = "/content/umls.duckdb"

# CSV 리더 공통 옵션 (인용 개행/초장문/트레일링 파이프 대응)
CSV_OPTS = "null_padding=true, strict_mode=false, parallel=false, max_line_size=20000000"


In [3]:
MRCONSO_IN = f"{UMLS_DIR}/MRCONSO.RRF"

total = kept = 0
# 'r'일기 모드로 열고, errors="ignore"를 통해 인코딩 오류가 발생해도 무시하고 계속 진행하라는 뜻
with open(MRCONSO_IN, "r", encoding="utf-8", errors="ignore") as fin, \
     open(MRCONSO_PREF_TMP, "w", encoding="utf-8") as fout:
    # 헤더
    fout.write("CUI,STR\n")
    for line in fin:
        parts = line.rstrip("\n").split("|")
        # 인덱스: 0=CUI, 1=LAT, 6=ISPREF('Y'/'N'), 12=TTY, 14=STR
        if len(parts) >= 15 and parts[1] == "ENG":
            isp = parts[6]
            tty = parts[12]
            if (isp == "Y") or (tty in ("PT","PN")):
                cui = parts[0]
                s   = (parts[14] or "").replace('"','""')
                fout.write(f'{cui},"{s}"\n')
                kept += 1
        total += 1
        if total % 2_000_000 == 0:
            print(f"[MRCONSO] read={total:,} kept={kept:,}")
print(f"[MRCONSO] DONE: read={total:,} kept={kept:,} -> {MRCONSO_PREF_TMP}")
gc.collect()


[MRCONSO] read=2,000,000 kept=608,231
[MRCONSO] read=4,000,000 kept=1,411,967
[MRCONSO] read=6,000,000 kept=2,301,283
[MRCONSO] read=8,000,000 kept=3,140,444
[MRCONSO] read=10,000,000 kept=4,243,715
[MRCONSO] read=12,000,000 kept=5,758,707
[MRCONSO] read=14,000,000 kept=7,103,909
[MRCONSO] read=16,000,000 kept=8,382,068
[MRCONSO] DONE: read=17,144,356 kept=9,171,982 -> /content/mrconso_pref_tmp.csv


0

In [4]:
MRDEF_IN = f"{UMLS_DIR}/MRDEF.RRF"

def looks_like_sab(s: str) -> bool:
    t = (s or "").strip()
    return 0 < len(t) <= 25 and t.replace("_","").isalnum() and t.upper() == t

def looks_like_sentence(s: str) -> bool:
    t = (s or "").strip()
    return len(t) >= 15 and any(c.islower() for c in t) and (" " in t or "." in t or ";" in t or "," in t)

total = kept = 0
with open(MRDEF_IN, "r", encoding="utf-8", errors="ignore") as fin, \
     open(MRDEF_TMP, "w", encoding="utf-8") as fout:
    # 헤더
    fout.write("CUI,DEF\n")
    for line in fin:
        p = line.rstrip("\n").split("|")
        if len(p) < 6:
            continue
        cui  = p[0]
        cand4 = p[4] if len(p) > 4 else ""  # 표준 DEF 위치
        cand5 = p[5] if len(p) > 5 else ""  # 당신 파일 DEF 위치

        # 자동 판별: p[4]가 SAB처럼, p[5]가 문장처럼 보이면 p[5] 선택
        if looks_like_sab(cand4) and looks_like_sentence(cand5):
            DEF = cand5
        elif looks_like_sentence(cand4):
            DEF = cand4
        else:
            # 문장형이 아니면 스킵 (데이터 깨짐 방지)
            continue

        DEF = DEF.replace('"','""')
        fout.write(f'{cui},"{DEF}"\n')
        kept += 1

print(f"[MRDEF] DONE: kept={kept:,} -> {MRDEF_TMP}")
gc.collect()


[MRDEF] DONE: kept=454,550 -> /content/mrdef_min_tmp.csv


0

In [5]:
con = duckdb.connect(DB_PATH)

# MRSTY (CUI, STY)
con.execute(f"""
CREATE OR REPLACE VIEW MRSTY AS
SELECT c0 AS CUI, c3 AS STY
FROM read_csv('{UMLS_DIR}/MRSTY.RRF',
  delim='|', header=false,
  columns={{'c0':'TEXT','c1':'TEXT','c2':'TEXT','c3':'TEXT','c4':'TEXT','c5':'TEXT','c6':'TEXT'}},
  auto_detect=false, sample_size=-1, {CSV_OPTS});
""")

# MRREL (CUI1, REL, CUI2, RELA, SAB) — 17칸 스키마(마지막은 트레일 흡수)
con.execute(f"""
CREATE OR REPLACE VIEW MRREL AS
SELECT CUI1, REL, CUI2, RELA, SAB FROM (
  SELECT
    c0  AS CUI1,
    c3  AS REL,
    c4  AS CUI2,
    c7  AS RELA,
    c10 AS SAB,
    c16 AS TRAIL
  FROM read_csv('{UMLS_DIR}/MRREL.RRF',
    delim='|', header=false,
    columns={{'c0':'TEXT','c1':'TEXT','c2':'TEXT','c3':'TEXT','c4':'TEXT',
              'c5':'TEXT','c6':'TEXT','c7':'TEXT','c8':'TEXT','c9':'TEXT',
              'c10':'TEXT','c11':'TEXT','c12':'TEXT','c13':'TEXT',
              'c14':'TEXT','c15':'TEXT','c16':'TEXT'}},
    auto_detect=false, sample_size=-1, {CSV_OPTS}
  )
);
""")

# 임시 CSV → 실제 테이블
con.execute("DROP TABLE IF EXISTS MRCONSO_ENG")
con.execute(f"CREATE TABLE MRCONSO_ENG AS SELECT * FROM read_csv_auto('{MRCONSO_PREF_TMP}', HEADER=true)")
con.execute("DROP TABLE IF EXISTS MRDEF_MIN")
con.execute(f"CREATE TABLE MRDEF_MIN  AS SELECT * FROM read_csv_auto('{MRDEF_TMP}', HEADER=true)")

# 카운트 확인
print("MRSTY      :", con.execute("SELECT COUNT(*) FROM MRSTY").fetchone()[0])
print("MRREL      :", con.execute("SELECT COUNT(*) FROM MRREL").fetchone()[0])
print("MRCONSO_ENG:", con.execute("SELECT COUNT(*) FROM MRCONSO_ENG").fetchone()[0])
print("MRDEF_MIN  :", con.execute("SELECT COUNT(*) FROM MRDEF_MIN").fetchone()[0])


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

MRSTY      : 3798191


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

MRREL      : 62908136
MRCONSO_ENG: 9171982
MRDEF_MIN  : 454550


In [6]:
# 질병/약물 STY 집합 (원하면 'Finding' 제거 가능)
DISEASE_STYS = (
  'Disease or Syndrome','Neoplastic Process','Mental or Behavioral Dysfunction',
  'Pathologic Function','Injury or Poisoning','Congenital Abnormality',
  'Acquired Abnormality','Finding'
)
DRUG_STYS = (
  'Pharmacologic Substance','Clinical Drug','Organic Chemical','Antibiotic',
  'Biomedical or Dental Material','Immunologic Factor','Hormone','Vitamin',
  'Hazardous or Poisonous Substance'
)

con.execute("CREATE OR REPLACE VIEW DISEASES AS SELECT DISTINCT CUI FROM MRSTY WHERE STY IN {}".format(DISEASE_STYS))
con.execute("CREATE OR REPLACE VIEW DRUGS    AS SELECT DISTINCT CUI FROM MRSTY WHERE STY IN {}".format(DRUG_STYS))

# 치료/적응증 RELA 후보 (contra 제외)
con.execute("""
CREATE OR REPLACE VIEW RELA_CAND AS
SELECT DISTINCT LOWER(RELA) AS rel
FROM MRREL
WHERE RELA IS NOT NULL AND (
  LOWER(RELA) LIKE '%treat%' OR
  LOWER(RELA) LIKE '%indicat%' OR
  LOWER(RELA) LIKE '%therapeutic%'
  -- 예방까지 포함하려면 아래 주석 해제
  -- OR LOWER(RELA) LIKE '%prevent%'
)
AND LOWER(RELA) NOT LIKE '%contra%';
""")

# 질병↔약물 양방향 정규화 (CUI1↔CUI2 양쪽 케이스 모두 커버)
con.execute("DROP VIEW IF EXISTS RELS")
con.execute("""
CREATE VIEW RELS AS
SELECT
  CASE WHEN d1.CUI IS NOT NULL AND k2.CUI IS NOT NULL THEN mr.CUI1 ELSE mr.CUI2 END AS disease_cui,
  CASE WHEN d1.CUI IS NOT NULL AND k2.CUI IS NOT NULL THEN mr.CUI2 ELSE mr.CUI1 END AS drug_cui,
  mr.RELA, mr.SAB
FROM MRREL mr
LEFT JOIN DISEASES d1 ON mr.CUI1 = d1.CUI
LEFT JOIN DRUGS    k1 ON mr.CUI1 = k1.CUI
LEFT JOIN DISEASES d2 ON mr.CUI2 = d2.CUI
LEFT JOIN DRUGS    k2 ON mr.CUI2 = k2.CUI
JOIN RELA_CAND rc ON LOWER(mr.RELA) = rc.rel
WHERE ((d1.CUI IS NOT NULL AND k2.CUI IS NOT NULL) OR (k1.CUI IS NOT NULL AND d2.CUI IS NOT NULL));
""")

print("RELS count:", con.execute("SELECT COUNT(*) FROM RELS").fetchone()[0])


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

RELS count: 27684


In [8]:
import pandas as pd

# 1. Pandas가 긴 텍스트를 자르지 않도록 설정 변경
pd.set_option('display.max_colwidth', None)

# 2. SQL 쿼리에서 SUBSTR 함수 제거
preview_df = con.execute("""
SELECT
  r.disease_cui,
  dn.STR AS disease_name,
  dd.DEF AS disease_description, -- SUBSTR(dd.DEF,1,300) 대신 dd.DEF 사용
  r.drug_cui,
  rn.STR AS drug_name,
  r.RELA, r.SAB
FROM RELS r
LEFT JOIN MRCONSO_ENG dn ON dn.CUI = r.disease_cui
LEFT JOIN MRDEF_MIN   dd ON dd.CUI = r.disease_cui
LEFT JOIN MRCONSO_ENG rn ON rn.CUI = r.drug_cui
LIMIT 5
""").fetchdf()

print(preview_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  disease_cui               disease_name  \
0    C0000768     Congenital Abnormality   
1    C0000768     Congenital Abnormality   
2    C0000768     Congenital abnormality   
3    C0000768     congenital abnormality   
4    C0000768  Abnormalities, Congenital   

                                                                           disease_description  \
0  Any abnormality, anatomical or biochemical, evident at birth or during the neonatal period.   
1  Any abnormality, anatomical or biochemical, evident at birth or during the neonatal period.   
2  Any abnormality, anatomical or biochemical, evident at birth or during the neonatal period.   
3  Any abnormality, anatomical or biochemical, evident at birth or during the neonatal period.   
4  Any abnormality, anatomical or biochemical, evident at birth or during the neonatal period.   

   drug_cui                 drug_name               RELA     SAB  
0  C0718043  sacrosidase (medication)  may_be_treated_by  MED-RT  
1  C0718043 

In [12]:
# --- 설정값 (원하면 조정) ---
PREVIEW_LIMIT = 5
SAB_FILTER = ("MED-RT","RXNORM")   # 소스 좁히면 훨씬 빨라짐. 비우려면 None 로.
MAKE_RELS_CACHE = True             # True면 RELS_T 캐시(한 번만 생성 후 재사용 추천)

# --- 선택: RELS를 캐시 테이블로 만들어두면 이후 미리보기가 즉시 응답 ---
if MAKE_RELS_CACHE:
    con.execute("DROP TABLE IF EXISTS RELS_T")
    con.execute(f"""
    CREATE TABLE RELS_T AS
    SELECT DISTINCT
      CASE WHEN d1.CUI IS NOT NULL AND k2.CUI IS NOT NULL THEN mr.CUI1 ELSE mr.CUI2 END AS disease_cui,
      CASE WHEN d1.CUI IS NOT NULL AND k2.CUI IS NOT NULL THEN mr.CUI2 ELSE mr.CUI1 END AS drug_cui,
      mr.RELA, mr.SAB
    FROM MRREL mr
    LEFT JOIN DISEASES d1 ON mr.CUI1 = d1.CUI
    LEFT JOIN DRUGS    k1 ON mr.CUI1 = k1.CUI
    LEFT JOIN DISEASES d2 ON mr.CUI2 = d2.CUI
    LEFT JOIN DRUGS    k2 ON mr.CUI2 = k2.CUI
    JOIN RELA_CAND rc ON LOWER(mr.RELA) = rc.rel
    WHERE ((d1.CUI IS NOT NULL AND k2.CUI IS NOT NULL) OR (k1.CUI IS NOT NULL AND d2.CUI IS NOT NULL))
    {f"AND mr.SAB IN ({', '.join([repr(x) for x in SAB_FILTER])})" if SAB_FILTER else ""}
    """)
    RELS_FROM = "RELS_T"
else:
    RELS_FROM = "RELS"

# --- 초저메모리 미리보기(휴리스틱 향상 버전) ---
preview_sql = f"""
WITH pairs AS (
  SELECT disease_cui, drug_cui, RELA, SAB
  FROM {RELS_FROM}
  {"WHERE SAB IN (" + ", ".join([repr(x) for x in SAB_FILTER]) + ")" if (SAB_FILTER and not MAKE_RELS_CACHE) else ""}
  LIMIT {PREVIEW_LIMIT}
)
SELECT
  p.disease_cui,

  -- 질병 대표명: 임상 용어 우선 -> 전부 대문자(약어) 회피 -> 길이>=6 -> 공백 포함 -> 짧은 순
  (SELECT STR
     FROM MRCONSO_ENG
    WHERE CUI = p.disease_cui
    ORDER BY
      CASE
        WHEN LOWER(STR) LIKE '% infection%' OR LOWER(STR) LIKE 'infection %'
          OR LOWER(STR) LIKE '% disease%'  OR LOWER(STR) LIKE '% syndrome%'
          OR STR LIKE '%(disorder)%'
        THEN 0 ELSE 1 END,
      CASE WHEN STR <> UPPER(STR) THEN 0 ELSE 1 END,
      CASE WHEN LENGTH(STR) >= 6 THEN 0 ELSE 1 END,
      CASE WHEN STR LIKE '% %' THEN 0 ELSE 1 END,
      LENGTH(STR), STR
    LIMIT 1) AS disease_name,

  -- 질병 정의: 영어스러운 문장 우선(관사/전치사/접속사) -> 긴 정의
  (SELECT DEF
     FROM MRDEF_MIN
    WHERE CUI = p.disease_cui
    ORDER BY
      CASE
        WHEN LOWER(DEF) LIKE '% the %' OR LOWER(DEF) LIKE '% of %'
          OR LOWER(DEF) LIKE '% and %' OR LOWER(DEF) LIKE '% or %'
        THEN 0 ELSE 1 END,
      LENGTH(DEF) DESC
    LIMIT 1) AS disease_description_300,

  p.drug_cui,

  -- 약물 대표명: 'product' 패널티 -> 전부 대문자 회피 -> 길이>=6 -> 공백 포함 -> 짧은 순
  (SELECT STR
     FROM MRCONSO_ENG
    WHERE CUI = p.drug_cui
    ORDER BY
      CASE WHEN LOWER(STR) LIKE '%product%' THEN 1 ELSE 0 END,
      CASE WHEN STR <> UPPER(STR) THEN 0 ELSE 1 END,
      CASE WHEN LENGTH(STR) >= 6 THEN 0 ELSE 1 END,
      CASE WHEN STR LIKE '% %' THEN 0 ELSE 1 END,
      LENGTH(STR), STR
    LIMIT 1) AS drug_name,

  p.RELA, p.SAB
FROM pairs p
"""

preview_lowmem = con.execute(preview_sql).fetchdf()
print(preview_lowmem)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  disease_cui                                disease_name  \
0    C5700101  Deficiency of alpha-glucosidase (disorder)   
1    C5235087                      Cryptococcus Infection   
2    C5203670                             SARS2 Infection   
3    C5203670                             SARS2 Infection   
4    C5203670                             SARS2 Infection   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [16]:
# RELS_FROM 확보 (RELS_T 있으면 그거 우선)
try:
    con.execute("SELECT 1 FROM RELS_T LIMIT 1")
    RELS_FROM = "RELS_T"
except:
    RELS_FROM = "RELS"

TARGET = "C0000768"  # Congenital Abnormality

# 영어 느낌의 정의를 우선해 CUI별 1개만 추리기 (임시 테이블)
con.execute("DROP TABLE IF EXISTS DEF_REP_EN")
con.execute("""
CREATE TEMP TABLE DEF_REP_EN AS
SELECT CUI, DEF FROM (
  SELECT
    CUI, DEF,
    ROW_NUMBER() OVER (
      PARTITION BY CUI
      ORDER BY
        -- 영어에서 흔한 단어(관사/전치사/접속사) 포함 우선
        CASE WHEN LOWER(DEF) LIKE '% the %' OR LOWER(DEF) LIKE '% of %'
                   OR LOWER(DEF) LIKE '% and %' OR LOWER(DEF) LIKE '% or %'
             THEN 0 ELSE 1 END,
        -- 비ASCII(라틴 외 문자) 포함하면 패널티 (regexp_matches 사용)
        CASE WHEN regexp_matches(DEF, '[^\\x00-\\x7F]') THEN 1 ELSE 0 END,
        -- 길이 긴 정의 선호
        LENGTH(DEF) DESC
    ) rn
  FROM MRDEF_MIN
  WHERE DEF IS NOT NULL AND DEF <> ''
) WHERE rn = 1
""")

# 영어 정의 버전으로 Congenital Abnormality 미리보기 (저메모리)
pretty_en = con.execute(f"""
WITH pairs AS (
  SELECT disease_cui, drug_cui, RELA, SAB
  FROM {RELS_FROM}
  WHERE disease_cui = '{TARGET}'
  ORDER BY drug_cui, RELA, SAB
  LIMIT 10
)
SELECT
  p.disease_cui,
  -- 질병 대표명: 임상어 우선 -> 전부 대문자 회피 -> 길이>=6 -> 공백 포함 -> 짧은 순
  (SELECT STR FROM MRCONSO_ENG
    WHERE CUI = p.disease_cui
    ORDER BY
      CASE
        WHEN LOWER(STR) LIKE '% infection%' OR LOWER(STR) LIKE 'infection %'
          OR LOWER(STR) LIKE '% disease%'  OR LOWER(STR) LIKE '% syndrome%'
          OR STR LIKE '%(disorder)%' THEN 0 ELSE 1
      END,
      CASE WHEN STR <> UPPER(STR) THEN 0 ELSE 1 END,
      CASE WHEN LENGTH(STR) >= 6 THEN 0 ELSE 1 END,
      CASE WHEN STR LIKE '% %' THEN 0 ELSE 1 END,
      LENGTH(STR), STR
    LIMIT 1) AS disease_name,

  -- 질병 정의: 영어스러운 문장 우선(관사/접속사) + 비ASCII 패널티 반영된 대표 정의
  (SELECT DEF FROM DEF_REP_EN WHERE CUI = p.disease_cui) AS disease_description_300,

  p.drug_cui,

  -- 약물 대표명: 'product' 패널티 -> 전부 대문자 회피 -> 길이>=6 -> 공백 포함 -> 짧은 순
  (SELECT STR FROM MRCONSO_ENG
    WHERE CUI = p.drug_cui
    ORDER BY
      CASE WHEN LOWER(STR) LIKE '%product%' THEN 1 ELSE 0 END,
      CASE WHEN STR <> UPPER(STR) THEN 0 ELSE 1 END,
      CASE WHEN LENGTH(STR) >= 6 THEN 0 ELSE 1 END,
      CASE WHEN STR LIKE '% %' THEN 0 ELSE 1 END,
      LENGTH(STR), STR
    LIMIT 1) AS drug_name,

  p.RELA, p.SAB
FROM pairs p
""").fetchdf()

print("\n== Pretty (English-only DEF heuristic) ==")
print(pretty_en)



== Pretty (English-only DEF heuristic) ==
  disease_cui                   disease_name  \
0    C0000768  Congenital anomaly (disorder)   
1    C0000768  Congenital anomaly (disorder)   

                                                                                                                                                                                                                                                                                    disease_description_300  \
0  Structural or functional abnormalities of the fetus. Note that this section comprises terms that describe abnormalities that are specific to the fetus or differ from the corresponding general terms. A term from anywhere in the Human Phenotype Ontology can be applied to a fetus if appropriate. []   
1  Structural or functional abnormalities of the fetus. Note that this section comprises terms that describe abnormalities that are specific to the fetus or differ from the corresponding general terms. A te

In [17]:
# 대괄호 [...] 로 끝나는 꼬리 텍스트 제거한 클린 뷰
con.execute("""
DROP VIEW IF EXISTS DEF_REP_EN_CLEAN;
CREATE TEMP VIEW DEF_REP_EN_CLEAN AS
SELECT
  CUI,
  -- 문장 끝의 [... ] 꼬리를 삭제
  regexp_replace(DEF, '\\s*\\[[^\\]]*\\]\\s*$', '') AS DEF
FROM DEF_REP_EN
""")


In [23]:
# RELS_FROM이 없으면 자동 선택
try:
    con.execute("SELECT 1 FROM RELS_T LIMIT 1")
    RELS_FROM = "RELS_T"
except:
    RELS_FROM = "RELS"

TARGET = "C0000768"  # 필요시 바꿔서 테스트

preview_clean = con.execute(f"""
WITH pairs AS (
  SELECT disease_cui, drug_cui, RELA, SAB
  FROM {RELS_FROM}
  WHERE disease_cui = '{TARGET}'
  ORDER BY drug_cui, RELA, SAB
  LIMIT 10
)
SELECT
  p.disease_cui,
  (SELECT STR FROM MRCONSO_ENG
    WHERE CUI = p.disease_cui
    ORDER BY
      CASE WHEN STR<>UPPER(STR) THEN 0 ELSE 1 END,
      CASE WHEN LENGTH(STR)>=6 THEN 0 ELSE 1 END,
      CASE WHEN STR LIKE '% %' THEN 0 ELSE 1 END,
      LENGTH(STR), STR
    LIMIT 1) AS disease_name,
  (SELECT DEF FROM DEF_REP_EN_CLEAN WHERE CUI=p.disease_cui) AS disease_description_300,
  p.drug_cui,
  (SELECT STR FROM MRCONSO_ENG
    WHERE CUI = p.drug_cui
    ORDER BY
      CASE WHEN LOWER(STR) LIKE '%product%' THEN 1 ELSE 0 END,
      CASE WHEN STR<>UPPER(STR) THEN 0 ELSE 1 END,
      CASE WHEN LENGTH(STR)>=6 THEN 0 ELSE 1 END,
      CASE WHEN STR LIKE '% %' THEN 0 ELSE 1 END,
      LENGTH(STR), STR
    LIMIT 1) AS drug_name,
  p.RELA, p.SAB
FROM pairs p
""").fetchdf()

print(preview_clean)


  disease_cui  disease_name                                                                                                                                                                                                                                                                                disease_description_300  drug_cui                drug_name               RELA     SAB
0    C0000768  Birth Defect  Structural or functional abnormalities of the fetus. Note that this section comprises terms that describe abnormalities that are specific to the fetus or differ from the corresponding general terms. A term from anywhere in the Human Phenotype Ontology can be applied to a fetus if appropriate.  C0718043  Sacrosidase (substance)  may_be_treated_by  MED-RT
1    C0000768  Birth Defect  Structural or functional abnormalities of the fetus. Note that this section comprises terms that describe abnormalities that are specific to the fetus or differ from the corresponding general terms. A 

In [30]:
con.execute("DROP VIEW IF EXISTS DEF_REP_EN_CLEAN_HTML")
con.execute(r"""
CREATE TEMP VIEW DEF_REP_EN_CLEAN_HTML AS
SELECT
  CUI,
  trim(
    regexp_replace(
      regexp_replace(
        regexp_replace(
          regexp_replace(
            regexp_replace(DEF,
              '<a[^>]*>.*?</a>', '', 'gi'   -- <a> ... </a> 전체 제거
            ),
            '<[^>]+>', '', 'gi'             -- 나머지 HTML 태그 제거
          ),
          'https?://\S+', '', 'gi'          -- URL 자체 제거
        ),
        '\[[^\]]*\]', '', 'g'               -- [ ... ] 인용 제거
      ),
      '\s+', ' ', 'g'                       -- 여분의 공백 정리
    )
  ) AS DEF
FROM DEF_REP_EN
""")


In [31]:
preview_clean = con.execute("""
SELECT
  p.disease_cui,
  dn.STR AS disease_name,
  SUBSTR(dd.DEF,1,300) AS disease_description,
  p.drug_cui,
  rn.STR AS drug_name,
  p.RELA, p.SAB
FROM PAIRS p
LEFT JOIN NAME_REP                 dn ON dn.CUI = p.disease_cui
LEFT JOIN DEF_REP_EN_CLEAN_HTML    dd ON dd.CUI = p.disease_cui
LEFT JOIN NAME_REP                 rn ON rn.CUI = p.drug_cui
LIMIT 5
""").fetchdf()
print(preview_clean)


  disease_cui                              disease_name                                                                                                                                                                                                                                                                                           disease_description  drug_cui               drug_name               RELA     SAB
0    C0009952                               Febrile Fit  Seizures that occur during a febrile episode. It is a common condition, affecting 2-5% of children aged 3 months to five years. An autosomal dominant pattern of inheritance has been identified in some families. The majority are simple febrile seizures (generally defined as generalized onset, single   C0282303    PHENobarbital Sodium          may_treat  MED-RT
1    C0018482                       Hemophilus diseases  Haemophilus is the name of a group of bacteria. There are several types of Haemophilus. They can cause di

In [32]:
SAVE = "/content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs_clean.csv"
con.execute(f"""
COPY (
  SELECT
    p.disease_cui,
    dn.STR AS disease_name,
    dd.DEF AS disease_description,
    p.drug_cui,
    rn.STR AS drug_name,
    p.RELA, p.SAB
  FROM PAIRS p
  LEFT JOIN NAME_REP                 dn ON dn.CUI = p.disease_cui
  LEFT JOIN DEF_REP_EN_CLEAN_HTML    dd ON dd.CUI = p.disease_cui
  LEFT JOIN NAME_REP                 rn ON rn.CUI = p.drug_cui
) TO '{SAVE}' WITH (HEADER, DELIMITER ',')
""")

print("✅ 링크 및 HTML 제거 버전 저장 완료:", SAVE)


✅ 링크 및 HTML 제거 버전 저장 완료: /content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs_clean.csv


# 상위 3개 결과 확인

In [33]:
import pandas as pd

pd.set_option('display.max_colwidth', None)         # 셀 내 텍스트 무제한
pd.set_option('display.max_columns', None)          # 모든 열 표시
pd.set_option('display.width', None)                # 가로 폭 제한 해제
pd.set_option('display.expand_frame_repr', False)   # 줄 바꿈 없이 한 줄로 표시

path = "/content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs.csv"

# CSV로드
df = pd.read_csv(path, encoding='utf-8', low_memory=False)

print(df.head(3).to_string(index=False))

disease_cui                       disease_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [34]:
import pandas as pd

# 표시 옵션: 글자/열 잘림 방지
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

path = "/content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs_clean.csv"

# 파일에서 상위 3행만 로드 (전체 로드 없음)
df_top3 = pd.read_csv(
    path,
    nrows=5,       # ★ 핵심: 처음 3행만 읽음
    dtype=str,     # 형 추론으로 인한 추가 스캔 방지
    encoding='utf-8',  # 인코딩 오류 시 'utf-8-sig' 또는 'cp949'로 변경
    engine='c'     # 기본값(c 엔진) 빠름; 문제가 있으면 'python'으로
)

print(df_top3.to_string(index=False))


disease_cui                       disease_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

# 정형 데이터 비정형 데이터로 변환

In [35]:
import pandas as pd
import os

IN_CSV  = "/content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs_clean.csv"
OUT_TXT = "/content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs_clean_unstructured.txt"

# 읽기 (NaN 방지)
df = pd.read_csv(IN_CSV, dtype=str, keep_default_na=False)

# 컬럼 이름 케이스/이름 변화에 대비한 매핑
col_map = {c.lower(): c for c in df.columns}
get = lambda *keys: next((col_map[k] for k in keys if k in col_map), None)

col_disease_name = get("disease_name")
col_desc         = get("disease_description", "disease_description_300")
col_drug_name    = get("drug_name")
col_rela         = get("rela","relationship","relationship attribute")
col_sab          = get("sab","source","source abbreviation")

# 확인
needed = [col_disease_name, col_desc, col_drug_name, col_rela, col_sab]
assert all(needed), f"필요 컬럼 누락: {needed}"

def clean(s: str) -> str:
    # 내부 줄바꿈은 보기 좋게 공백으로 변환
    return (s or "").replace("\r\n", " ").replace("\n", " ").strip()

blocks = []
for _, row in df.iterrows():
    block = (
        f"disease name: {clean(row[col_disease_name])}\n"
        f"disease description: {clean(row[col_desc])}\n"
        f"drug name: {clean(row[col_drug_name])}\n"
        f"relationship attribute: {clean(row[col_rela])}\n"
        f"source abbreviation: {clean(row[col_sab])}"
    )
    blocks.append(block)

# 블록 사이에 빈 줄 1개(=줄바꿈 2번)로 구분
text = "\n\n".join(blocks)

# 저장
os.makedirs(os.path.dirname(OUT_TXT), exist_ok=True)
with open(OUT_TXT, "w", encoding="utf-8") as f:
    f.write(text)

print("✅ 저장:", OUT_TXT, "| 블록 수:", len(blocks))


✅ 저장: /content/drive/MyDrive/DILAB/MARS/UMLS/mapped_disease_drug_with_defs_clean_unstructured.txt | 블록 수: 27682
